In [62]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import ezodf
import numpy as np

In [142]:
### Functions 

def ods_to_df(doc_file_path):
    # convert the first sheet to a pandas.DataFrame
    
    doc = ezodf.opendoc(doc_file_path)
    
    sheet = doc.sheets[0]
    df_dict = {}
    for i, row in enumerate(sheet.rows()):
        # row is a list of cells
        # assume the header is on the first row
        if i == 0:
            # columns as lists in a dictionary
            df_dict = {cell.value:[] for cell in row}
            # create index for the column headers
            col_index = {j:cell.value for j, cell in enumerate(row)}
            continue
        for j, cell in enumerate(row):
            # use header instead of column index
            df_dict[col_index[j]].append(cell.value)
    # and convert to a DataFrame
    df = pd.DataFrame(df_dict)
    
    return(df)

In [143]:
# Variables to Read in Data

mypath = '/Users/diego/Desktop/Projects_Code/marginal_value_ei/Data/meter_data'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

names = ['unnamed: 0',
 'alumbrado publico',
 'cargos varios',
 'comercializacion',
 'csmo. energía:',
 'energia (kwh)',
 'fecha factura ant.:',
 'fecha factura:',
 'iva'        ,
 'regulacion ine',
 'subsidio alumb. p�blico menor 150 kwh',
 'subsidio comercializacion menor 150 kwh',
 'subsidio consumo menor 150 kwh',
 'subsidio a jubilados con consumo menor 150kwh',
 'tarifa:',
 'total']

In [149]:
# Read Data

dfs = []
for file_name in onlyfiles[1:]:
    
    split_name = file_name.split('.')[0]
    split_type = file_name.split('.')[1]

    if split_type == 'csv':
        data = pd.read_csv(mypath + '/' + file_name) 
        data.columns = map(str.lower, data.columns)
    else: 
        data = ods_to_df(mypath+ '/' + file_name)
        data.columns.values[0] = "unnamed: 0"
        data.columns = map(str.lower, data.columns)

    # Identify missing columns, add NANs new data frame, append
    data_column_names = list(data.columns)
    missing_column_list = np.setdiff1d(names,data_column_names)

    for missing_column in range(0,len(missing_column_list)):
        data[missing_column_list[missing_column]] = np.nan

    # Create DF and add ID    
    
    nis = file_name.split('_')
    data_ordered = data[names]
    data_ordered['nis'] = nis[0]

    dfs.append(data_ordered)

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [152]:
df_data = pd.concat(dfs)

In [154]:
df_data.tail()

,unnamed: 0,alumbrado publico,cargos varios,comercializacion,csmo. energía:,energia (kwh),fecha factura ant.:,fecha factura:,iva,regulacion ine,subsidio alumb. p�blico menor 150 kwh,subsidio comercializacion menor 150 kwh,subsidio consumo menor 150 kwh,subsidio a jubilados con consumo menor 150kwh,tarifa:,total,nis
8,8.0,100.03,NaN,139.29,863 kWh,"6,052.29",09-11-2016,12-12-2016,953.18,62.92,NaN,NaN,NaN,NaN,T-1,"7,307.71",3129115
9,9.0,51.07,NaN,29.15,142 kWh,635.27,10-10-2016,09-11-2016,108.4,7.15,NaN,NaN,NaN,NaN,T-1,831.04,3129115
10,10.0,99.21,NaN,138.17,534 kWh,"3,713.86",08-09-2016,10-10-2016,598.61,39.51,NaN,NaN,NaN,NaN,T-1,"4,589.36",3129115
11,11.0,98.81,NaN,137.60,420 kWh,"2,908.70",09-08-2016,08-09-2016,476.48,31.45,NaN,NaN,NaN,NaN,T-1,"3,653.04",3129115
0,0.0,13.61,NaN,30.08,NaN,NaN,02-02-2018,10-02-2018,NaN,0.44,NaN,NaN,NaN,NaN,T-0,44.13,3250855


In [ ]:
df_data.to_csv('/Users/diego/Desktop/Projects_Code/marginal_value_ei/Data/meter_data/df_data.csv')